In [ ]:
pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.40.0.3-py2.py3-none-any.whl size=177694727 sha256=8418317e90d28447fff430d932874e5660b5bc0ac5ce23b02a5581055860f2ea
  Stored in directory: /root/.cache/pip/wheels/9a/54/b6/c9ab3e71309ef0000bbe39e715020dc151bbfc557784b7f4c9
Successfully built h2o


In [ ]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Initialize H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,17 mins 00 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_unknownUser_npm6mq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.155 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
games_df = pd.read_csv("/content/drive/My Drive/DS320/games.csv")
recommendations_df = pd.read_csv("/content/drive/My Drive/DS320/recommendations.csv")
data = games_df.merge(recommendations_df, on="app_id")

In [ ]:
# Randomly select 1000 rows from the data DataFrame
sampled_data = data.sample(n=1000, random_state=42)

# Reset the index of the new DataFrame
sampled_data.reset_index(drop=True, inplace=True)

In [ ]:
sampled_data.columns

Index(['app_id', 'title', 'date_release', 'win', 'mac', 'linux', 'rating',
       'positive_ratio', 'user_reviews', 'price_final', 'price_original',
       'discount', 'steam_deck', 'helpful', 'funny', 'date', 'is_recommended',
       'hours', 'user_id', 'review_id'],
      dtype='object')

In [ ]:
# Preprocess categorical feature
sampled_data['rating'] = sampled_data['rating'].replace({
    'Very Positive': 3,
    'Overwhelmingly Positive': 2,
    'Mostly Positive': 1,
    'Mixed': 0,
    'Mostly Negative': -1
})


In [ ]:
print(sampled_data.columns)

Index(['app_id', 'title', 'date_release', 'win', 'mac', 'linux', 'rating',
       'positive_ratio', 'user_reviews', 'price_final', 'price_original',
       'discount', 'steam_deck', 'helpful', 'funny', 'date', 'is_recommended',
       'hours', 'user_id', 'review_id'],
      dtype='object')


In [ ]:
num_rows = sampled_data.shape[0]

print("Total number of rows:", num_rows)

Total number of rows: 1000


In [ ]:
sampled_data.head(100)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck,helpful,funny,date,is_recommended,hours,user_id,review_id
0,860510,Little Nightmares II,2021-02-10,True,False,False,3,94,21303,29.99,29.99,0.0,True,0,0,2021-02-21,True,7.0,2025914,6438113
1,730,Counter-Strike: Global Offensive,2012-08-21,True,True,True,3,88,6941137,14.99,0.00,0.0,True,3,0,2019-06-30,True,575.1,1953358,4796170
2,238960,Path of Exile,2013-10-23,True,True,False,3,87,196492,0.00,0.00,0.0,True,0,0,2016-11-24,True,879.7,4120175,501722
3,823500,BONEWORKS,2019-12-10,True,False,False,3,92,31004,29.99,29.99,0.0,True,0,0,2019-12-14,True,19.7,4676425,7276704
4,438100,VRChat,2017-02-01,True,False,False,1,71,191010,0.00,0.00,0.0,True,0,0,2022-07-25,True,12.4,2261334,5173684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4700,Total War: MEDIEVAL II – Definitive Edition,2007-11-27,True,True,True,2,95,20077,24.99,24.99,0.0,True,0,0,2021-04-21,True,111.1,3656961,9492946
96,236850,Europa Universalis IV,2013-08-13,True,True,True,3,86,78089,39.99,39.99,0.0,True,0,0,2021-05-30,True,0.1,5529097,6110713
97,1286830,STAR WARS™: The Old Republic™,2020-07-21,True,False,False,3,89,49272,0.00,0.00,0.0,True,9,0,2020-07-23,False,17.0,4198729,3000794
98,1599340,Lost Ark,2022-02-11,True,False,False,1,71,189423,0.00,0.00,0.0,True,0,0,2021-06-14,True,542.9,5280887,3371578


In [ ]:

#data.dropna()
#data.isna().sum()
len(data)

sampled_data = sampled_data.drop(columns=['title', 'app_id'])
sampled_data['date_release'] = pd.to_datetime(sampled_data['date_release']).apply(lambda x: x.year)
sampled_data['date'] = pd.to_datetime(sampled_data['date']).apply(lambda x: x.year)

In [ ]:
print(sampled_data.columns)

Index(['date_release', 'win', 'mac', 'linux', 'rating', 'positive_ratio',
       'user_reviews', 'price_final', 'price_original', 'discount',
       'steam_deck', 'helpful', 'funny', 'date', 'is_recommended', 'hours',
       'user_id', 'review_id'],
      dtype='object')


In [ ]:
# Remove spaces from column names
sampled_data.columns = [col.replace(" ", "") for col in sampled_data.columns]

# Display the updated column names
print(sampled_data.columns)

Index(['date_release', 'win', 'mac', 'linux', 'rating', 'positive_ratio',
       'user_reviews', 'price_final', 'price_original', 'discount',
       'steam_deck', 'helpful', 'funny', 'date', 'is_recommended', 'hours',
       'user_id', 'review_id'],
      dtype='object')


In [ ]:
len(sampled_data)

1000

In [ ]:
# Split dataset into training and testing sets
train_data, temp_data = train_test_split(sampled_data, test_size=0.5, random_state=42)
test_data, val_data = train_test_split(temp_data, test_size=0.6, random_state=42)

In [ ]:
# Convert pandas dataframes to H2O dataframes
h2o_train_data = h2o.H2OFrame(train_data)
h2o_test_data = h2o.H2OFrame(test_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Identify the target column
y = "price_final"
x = train_data.drop(columns=[y]).columns.tolist()

In [ ]:
# Run H2O AutoML
aml = H2OAutoML(max_models=3, seed=1)
aml.train(x=x, y=y, training_frame=h2o_train_data)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_1_AutoML_2_20230427_61637


Model Summary: 
    number_of_trees
--  -----------------
    87

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 2.5888249015360234
RMSE: 1.6089825671945683
MAE: 0.564238942861557
RMSLE: 0.32481735068854706
Mean Residual Deviance: 2.5888249015360234

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 6.194361467006234
RMSE: 2.488847417381434
MAE: 0.9661725319623947
RMSLE: NaN
Mean Residual Deviance: 6.194361467006234

Cross-Validation Metrics Summary: 
                        mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
mae                     0.966172  0.26997     1.13802       1.26452       0.650327      1.0641        0.713893
mean_residual_deviance  6.19436   2.41291     6.56663       9.79472       3.08845       5.47368       6.04833
mse                     6.19436   2.41291     6.56663       9.79472       3.08845       5.47368       6.04833
r2                      0.98029   0.00724035  0.981226      0.969926      0.990348      0.979832      0.98012
residual_deviance       6.19436   2.41291     6.56663       9.79472       3.08845       5.47368       6.04833
rmse                    2.4497    0.49157     2.56254       3.12965       1.7574        2.33959       2.45933
rmsle                   0.391688  0.0802266   nan           nan           0.427203      0.448027      0.299834

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2023-04-27 06:16:42  4.071 sec   0                  29.5939          23.7873         875.798
    2023-04-27 06:16:42  4.105 sec   5                  7.56172          5.85418         57.1797
    2023-04-27 06:16:42  4.113 sec   10                 3.63512          2.65716         13.2141
    2023-04-27 06:16:42  4.128 sec   15                 2.39604          1.44136         5.74099
    2023-04-27 06:16:42  4.143 sec   20                 2.16007          1.14685         4.66592
    2023-04-27 06:16:42  4.326 sec   25                 1.98478          0.926603        3.93934
    2023-04-27 06:16:42  4.343 sec   30                 1.93587          0.833913        3.74761
    2023-04-27 06:16:42  4.352 sec   35                 1.91557          0.798297        3.6694
    2023-04-27 06:16:42  4.368 sec   40                 1.8576           0.773961        3.45067
    2023-04-27 06:16:42  4.385 sec   45                 1.80484          0.745635        3.25746
    2023-04-27 06:16:42  4.400 sec   50                 1.7833           0.719344        3.18016
    2023-04-27 06:16:42  4.413 sec   55                 1.77007          0.705632        3.13313
    2023-04-27 06:16:42  4.427 sec   60                 1.73083          0.647428        2.99578
    2023-04-27 06:16:42  4.441 sec   65                 1.69479          0.620052        2.87231
    2023-04-27 06:16:42  4.455 sec   70                 1.6637           0.602175        2.76789
    2023-04-27 06:16:42  4.468 sec   75                 1.64854          0.567383        2.71768
    2023-04-27 06:16:42  4.523 sec   80                 1.63076          0.582014        2.65938
    2023-04-27 06:16:42  4.807 sec   85                 1.61568          0.559414        2.61042
    2023-04-27 06:16:42  4.823 sec   87                 1.60898          0.564239        2.58882

Variable Importances: 
variable              relative_importance    scaled_importance    percentage
--------------------  ---------------------  -------------------  ------------
price_original        171400                 1                    0.887921
discount              13236.7                0.077227    

In [ ]:
# Get the best model
best_model = aml.leader

In [ ]:
# Get the leaderboard of models
leaderboard = aml.leaderboard

# Convert the leaderboard to a pandas DataFrame
leaderboard_df = leaderboard.as_data_frame()

# Get the top 3 models as a new DataFrame
top_3_models_df = leaderboard_df.head(3)

# Print the top 3 models DataFrame
print(top_3_models_df)

                                            model_id      rmse        mse  \
0                  XGBoost_1_AutoML_2_20230427_61637  2.488847   6.194361   
1  StackedEnsemble_BestOfFamily_1_AutoML_2_202304...  2.570725   6.608627   
2                      GLM_1_AutoML_2_20230427_61637  3.582393  12.833539   

        mae  rmsle  mean_residual_deviance  
0  0.966173    NaN                6.194361  
1  1.018275    NaN                6.608627  
2  2.045400    NaN               12.833539  


In [ ]:
h2o_val_data = h2o.H2OFrame(val_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Predict on the test set and validation
y_pred_test = best_model.predict(h2o_test_data).as_data_frame().values.flatten()
y_pred_val = best_model.predict(h2o_val_data).as_data_frame().values.flatten()


xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [ ]:
# Evaluate the model
mse_test = mean_squared_error(test_data[y], y_pred_test)
mse_val = mean_squared_error(val_data[y], y_pred_val)
print(f"Mean Squared Error (Test Set): {mse_test}")
print(f"Mean Squared Error (Validation Set): {mse_val}")

Mean Squared Error (Test Set): 3.0571726184211183
Mean Squared Error (Validation Set): 3.1001021330536123


##Using Mostly Correlated Features

In [ ]:
# Round 2
# Dataset with Selected Features
feature_data = sampled_data[['price_final', 'price_original', 'date_release', 'discount', 'linux',
                'mac', 'date', 'user_reviews']]

In [ ]:
print(feature_data.columns)

Index(['price_final', 'price_original', 'date_release', 'discount', 'linux',
       'mac', 'date', 'user_reviews'],
      dtype='object')


In [ ]:
# Split dataset into training and testing sets
train_data, temp_data = train_test_split(feature_data, test_size=0.5, random_state=42)
test_data, val_data = train_test_split(temp_data, test_size=0.6, random_state=42)

In [ ]:
# Convert pandas dataframes to H2O dataframes
h2o_train_data = h2o.H2OFrame(train_data)
h2o_test_data = h2o.H2OFrame(test_data)
h2o_val_data = h2o.H2OFrame(val_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Identify the target column
y = "price_final"
x = train_data.drop(columns=[y]).columns.tolist()

In [ ]:
# Run H2O AutoML
aml = H2OAutoML(max_models=3, seed=1)
aml.train(x=x, y=y, training_frame=h2o_train_data)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),2/3
# GBM base models (used / total),0/1
# XGBoost base models (used / total),1/1
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [ ]:
# Get the best model
best_model = aml.leader

In [ ]:
# Get the leaderboard of models
leaderboard = aml.leaderboard

# Convert the leaderboard to a pandas DataFrame
leaderboard_df = leaderboard.as_data_frame()

# Check if there are at least 3 models in the leaderboard
if len(leaderboard_df) >= 3:
    # Get the top 3 models as a new DataFrame
    top_3_models_df = leaderboard_df.head(3)
else:
    # Get all models as a new DataFrame
    top_3_models_df = leaderboard_df.copy()

# Print the top 3 models DataFrame
print(top_3_models_df)

                                            model_id      rmse        mse  \
0  StackedEnsemble_BestOfFamily_1_AutoML_5_202304...  2.566923   6.589093   
1                  XGBoost_1_AutoML_5_20230427_63619  2.707011   7.327910   
2                      GLM_1_AutoML_5_20230427_63619  3.211712  10.315095   

        mae  rmsle  mean_residual_deviance  
0  1.255446    NaN                6.589093  
1  1.268355    NaN                7.327910  
2  1.668830    NaN               10.315095  


In [ ]:
# Predict on the test set
y_pred_test = best_model.predict(h2o_test_data).as_data_frame().values.flatten()
y_pred_val = best_model.predict(h2o_val_data).as_data_frame().values.flatten()


stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [ ]:
# Evaluate the model
mse_test = mean_squared_error(test_data[y], y_pred_test)
mse_val = mean_squared_error(val_data[y], y_pred_val)
print(f"Mean Squared Error (Test Set): {mse_test}")
print(f"Mean Squared Error (Validation Set): {mse_val}")

Mean Squared Error (Test Set): 2.2962831780374398
Mean Squared Error (Validation Set): 3.2720797414469582


In [ ]:
# Clean up
h2o.cluster().shutdown()